> ### Reconhecedor de padrões baseado em LZW

#### Pré-processamento das imagens

##### 1. Retirada do cabeçalho dos dados

In [23]:
import os

In [48]:
# Diretório de origem
diretorio_origem = "/home/franklin/Documents/academic/Eng. de Computação/P8/Intro. à Teo. da Informação/Projeto/parte-2/encode-decode-lzw/db/orl_faces/"

# Diretório de destino
diretorio_destino = "/home/franklin/Documents/academic/Eng. de Computação/P8/Intro. à Teo. da Informação/Projeto/parte-2/encode-decode-lzw/db/orl_faces_sc"

In [25]:
# Percorre todas as subpastas no diretório de origem
for subpasta in os.listdir(diretorio_origem):
    caminho_subpasta = os.path.join(diretorio_origem, subpasta)

    # Cria a subpasta correspondente no diretório de destino
    caminho_destino_subpasta = os.path.join(diretorio_destino, subpasta)
    os.makedirs(caminho_destino_subpasta, exist_ok=True)

    # Percorre os arquivos de imagem na subpasta atual
    for arquivo in os.listdir(caminho_subpasta):
        if arquivo.endswith(".pgm"):
            caminho_arquivo_origem = os.path.join(caminho_subpasta, arquivo)

            # Abre o arquivo de origem em modo binário
            with open(caminho_arquivo_origem, "rb") as arquivo_origem:
                dados = arquivo_origem.read()

            # Remove as três primeiras linhas
            novos_dados = dados.split(b"\n", 3)[-1]

            # Cria o caminho do arquivo de destino
            caminho_arquivo_destino = os.path.join(caminho_destino_subpasta, arquivo)

            # Salva os novos dados no arquivo de destino em modo binário
            with open(caminho_arquivo_destino, "wb") as arquivo_destino:
                arquivo_destino.write(novos_dados)

#### Geração dos modelos

##### 1. Divisão aleatória do banco entre instâncias de "treinamento" e "teste" - **ocorre a cada novo valor de k**

In [42]:
import os
import shutil
import random

In [43]:
# Diretório de origem
diretorio_origem = "/home/franklin/Documents/academic/Eng. de Computação/P8/Intro. à Teo. da Informação/Projeto/parte-2/encode-decode-lzw/db/orl_faces_sc"

# Diretório de destino para treinamento
diretorio_destino_treinamento = "/home/franklin/Documents/academic/Eng. de Computação/P8/Intro. à Teo. da Informação/Projeto/parte-2/encode-decode-lzw/db/orl_faces_div/train"

# Diretório de destino para teste
diretorio_destino_teste = "/home/franklin/Documents/academic/Eng. de Computação/P8/Intro. à Teo. da Informação/Projeto/parte-2/encode-decode-lzw/db/orl_faces_div/test"

# Porcentagem de imagens para treinamento (90%)
porcentagem_treinamento = 0.9

In [51]:
# Exclui os diretórios de destino, se já existirem
if os.path.exists(diretorio_destino_treinamento):
    shutil.rmtree(diretorio_destino_treinamento)
if os.path.exists(diretorio_destino_teste):
    shutil.rmtree(diretorio_destino_teste)

In [52]:
# Percorre todas as subpastas no diretório de origem
for subpasta in os.listdir(diretorio_origem):
    caminho_subpasta_origem = os.path.join(diretorio_origem, subpasta)

    # Cria a subpasta correspondente no diretório de destino para treinamento
    caminho_subpasta_destino_treinamento = os.path.join(diretorio_destino_treinamento, subpasta)
    os.makedirs(caminho_subpasta_destino_treinamento)

    # Cria a subpasta correspondente no diretório de destino para teste
    caminho_subpasta_destino_teste = os.path.join(diretorio_destino_teste, subpasta)
    os.makedirs(caminho_subpasta_destino_teste)

    # Lista os arquivos na subpasta atual
    arquivos = os.listdir(caminho_subpasta_origem)
    random.shuffle(arquivos)  # Embaralha a ordem dos arquivos

    # Calcula o número de arquivos para treinamento e teste
    num_arquivos_treinamento = int(len(arquivos) * porcentagem_treinamento)
    num_arquivos_teste = len(arquivos) - num_arquivos_treinamento

    # Divide os arquivos entre treinamento e teste
    arquivos_treinamento = arquivos[:num_arquivos_treinamento]
    arquivos_teste = arquivos[num_arquivos_treinamento:]

    # Move os arquivos para as respectivas pastas de treinamento e teste
    for arquivo in arquivos_treinamento:
        caminho_arquivo_origem = os.path.join(caminho_subpasta_origem, arquivo)
        caminho_arquivo_destino = os.path.join(caminho_subpasta_destino_treinamento, arquivo)
        shutil.copyfile(caminho_arquivo_origem, caminho_arquivo_destino)

    for arquivo in arquivos_teste:
        caminho_arquivo_origem = os.path.join(caminho_subpasta_origem, arquivo)
        caminho_arquivo_destino = os.path.join(caminho_subpasta_destino_teste, arquivo)
        shutil.copyfile(caminho_arquivo_origem, caminho_arquivo_destino)

##### 2. Construção dos modelos para cada classe

In [16]:
def lzw_encode(input_file, max_dict_size, type_process, n_loop):
    if type_process == 0 and n_loop == 0:
        # Inicializa o dicionário LZW com todos os caracteres ASCII possíveis
        dict_size = 256
        dictionary = {bytes([i]): i for i in range(dict_size)}
        current_sequence = bytes()
    else if type_process == 0 and n_loop > 0:
        dictionary = dictionary
    
    with open(input_file, "rb") as file, open("output.bin", "wb") as output_file:
        # Codifica cada símbolo da entrada
        while True:
            symbol = file.read(1)
            if not symbol:
                break
                
            # Acrescenta o próximo símbolo à sequência atual
            next_sequence = current_sequence + symbol # BYTE
            # Se a próxima sequência estiver no dicionário, continua acrescentando símbolos
            # até chegar a uma sequência que não esteja no dicionário
            if next_sequence in dictionary:
                current_sequence = next_sequence
            else:
                # Codifica a sequência atual e adiciona a nova sequência ao dicionário
                encoded_sequence = dictionary[current_sequence]
                output_file.write(encoded_sequence.to_bytes(2, byteorder="big"))
                
                if type_process == 0:
                    if dict_size < 2 ** max_dict_size:
                        dictionary[next_sequence] = dict_size
                        dict_size += 1
                # Começa uma nova sequência com o último símbolo lido
                current_sequence = symbol
        if type_process == 0:
            # Escreve o último código LZW no arquivo de saída
            if current_sequence:
                encoded_sequence = dictionary[current_sequence]
                output_file.write(encoded_sequence.to_bytes(2, byteorder="big"))

In [ ]:
cont = 0
for i in list_dict:
    lzw_encode()
    cont += 1